# Architecture analysis at node scale

## 0. Module requierement

### 0.1. import standard module python

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain
from collections import OrderedDict, defaultdict

### 0.2 import openalea modules

In [12]:
from openalea.mtg import algo, traversal

### 0.2. import strawberry modules

In [13]:
import openalea.strawberry #import openalea.strawberry module
from openalea.strawberry.Import_mtgfile import import_mtgfile
from openalea.strawberry.Import_mtgfile import plant_number_by_varieties
from openalea.strawberry.variables import extract_at_module_scale
from openalea.strawberry.analysis import occurence_module_order_along_time, pointwisemean_plot, crowntype_distribution

## 1. Import  and read mtg files

In [14]:
g = import_mtgfile(filename= "Gariguette")

In [15]:
roots = g.component_roots_at_scale(g.root, scale=2)
roots

[2,
 74,
 147,
 264,
 327,
 383,
 445,
 505,
 562,
 636,
 817,
 914,
 1051,
 1202,
 1322,
 1427,
 1534,
 1621,
 1710,
 1948,
 2068,
 2168,
 2239,
 2442,
 2604,
 2679,
 2791,
 2897,
 2988,
 3076,
 3195,
 3325,
 3442,
 3554,
 3628,
 3733,
 3880,
 4125,
 4243,
 4311,
 4477,
 4557,
 4654,
 4769,
 5079,
 5183,
 5329,
 5512,
 5655,
 5726,
 5861,
 6055,
 6170,
 6294]

In [17]:
def apparent_axis(g, vid):
    visibles = g.property('visible')
    v = vid
    while v is not None:
        yield v
        vtx = v; v = None
        for vid in g.children(vtx):
            if (vid in visibles) and (not is_axis_root(g, vid)):
                v = vid

In [21]:
trunks = [ list(chain(*[(v for v in algo.axis(g,m, scale=3) if g.label(v) in ('F', 'f')) for m in apparent_axis(g, r)])) for r in roots]

In [23]:
vertex_ids = [[str(rank) for rank, v in enumerate(trunk)] for trunk in trunks]
vertex_ids

[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'],
 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'],
 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13'],
 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 ['0', '1', '2', '3', '4', '5', '6', '7', '8'],
 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
 ['0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18'],
 ['0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14'],
 ['0', '1', '2', '3', '4', '5', '6'],
 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'],
 ['0', '1', '2', '3', '4', '5', '6', '7'

In [25]:
def branching(g, v):
    sid = g.Successor(v)
    if g.label(sid) in ('bt', 'ht', 'HT'):
        return None
    
    kids = [c for c in g.children(v) if g.edge_type(c) =='+' and g.label(c) == 'F' ]
    if not kids:
        return None
    
    return kids[0]

def branching_type(vid, g=g):
    """ Returns the type of branching
    
    :Algorithms:
    
    if module is visible:
        - branch crown (complex ramification):6
        - inflorescence : 7

    if module is invisible:
        - stolon (s): 1,
        - vegetative bud(bt, at stage None, 17,18,19):2,
        - initiated bud, (bt, at stage A):3,
        - aborted or roten or dried bud: 4
        - floral bud(ht):5
    """

    cpx = g.complex(vid)
    nxid = g.node(cpx) 
    if nxid.visible:
        if g.label(vid) == 'HT':
            return 7
        else:
            return 6
    
    # select s, ht, et bt
    for cid in nxid.components():
        label = cid.label
        if label in ('s', 'ht', 'bt'):
            if label == 's':
                return 1
            elif label == 'bt':
                stage = cid.Stade
                if stage in (None, '17', '18', '19'):
                    return 2
                elif stage == 'A':
                    return 3
                elif stage in ('pourri', 'aborted', 'dried'):
                    return 4
            elif label  == 'ht':
                return 5
    else:
        print 'ERROR: ', cpx, nxid.complex().Genotype, nxid.properties()

# Extract on a vertex the tree of modules
def module_tree(v, g):
    visibles = g.property('visible')
    cpx = g.complex(v)
    return [m for m in traversal.pre_order2(g, cpx) if m in visibles]


In [26]:
def module_axis(v, g=g):
    cpx = g.complex(v)
    return list(apparent_axis(g, cpx))

def nb_modules_axis(v, g=g):
    return len(module_axis(v, g=g))

def nb_visible_leaves_axis(v, g=g):
    return sum(nb_visible_leaves(m, g) for m in module_axis(v, g=g))

def nb_foliar_primordia_axis(v, g=g):
    return sum(nb_foliar_primordia(m, g) for m in module_axis(v, g=g))

def nb_total_leaves_axis(v, g=g):
    return sum(nb_total_leaves(m, g) for m in module_axis(v, g=g))

def nb_open_flowers_axis(v, g=g):
    return sum(nb_open_flowers(m, g) for m in module_axis(v, g=g))

def nb_aborted_flowers_axis(v, g=g):
    return sum(nb_aborted_flowers(m, g) for m in module_axis(v, g=g))

def nb_total_flowers_axis(v, g=g):
    return sum(nb_total_flowers(m, g) for m in module_axis(v, g=g))

def nb_vegetative_buds_axis(v, g=g):
    return sum(nb_vegetative_buds(m, g) for m in module_axis(v, g=g))

def nb_initiated_buds_axis(v, g=g):
    return sum(nb_initiated_buds(m, g) for m in module_axis(v, g=g))

def nb_floral_buds_axis(v, g=g):
    return sum(nb_floral_buds(m, g) for m in module_axis(v, g=g))

def nb_stolons_axis(v, g=g):
    return sum(nb_stolons(m, g) for m in module_axis(v, g=g))


####################################################
def nb_total_module_tree(v, g=g):
    return len(module_tree(v, g))

def nb_branching_tree(v, g=g):
    return sum(1 for m in module_tree(v, g) if is_axis_root(g, m))

def nb_extension_tree(v,g=g):
    return sum(1 for m in module_tree(v,g) if not is_axis_root(g, m))

def nb_visible_leaves_tree(v, g=g):
    return sum(nb_visible_leaves(m,g) for m in module_tree(v, g))

def nb_primordia_leaves_trees(v, g=g):
    return sum(nb_foliar_primordia(m,g) for m in module_tree(v, g))

def nb_total_leaves_trees(v, g=g):
    return sum(nb_total_leaves(m, g) for m in module_tree(v, g))

def nb_vegetative_buds_tree(v, g=g):
    return sum(nb_vegetative_buds(m, g) for m in module_tree(v, g))

def nb_initiated_buds_tree(v, g=g):
    return sum(nb_initiated_buds(m, g) for m in module_tree(v, g))

def nb_floral_buds_tree(v, g=g):
    return sum(nb_floral_buds(m, g) for m in module_tree(v, g))

def nb_inflorescences_tree(v, g=g):
    return sum(nb_inflorescences(m, g) for m in module_tree(v, g))

def nb_open_flowers_tree(v, g=g):
    return sum(nb_open_flowers(m, g) for m in module_tree(v, g))

def nb_aborted_flowers_tree(v , g=g):
    return sum(nb_aborted_flowers(m, g=g) for m in module_tree(v, g))

def nb_total_flowers_tree(v, g=g):
    return sum(nb_total_flowers(m, g=g) for m in module_tree(v, g))

def nb_stolon_tree(v, g=g):
    return sum(nb_stolons(m, g) for m in module_tree(v, g))

def nb_inflorescences_tree(v, g=g):
    return sum(nb_inflorescence (m, g) for m in module_tree(v, g))

##################################################
def genotype_axis(v, g=g):
    return genotype(g.complex(v), g)
def date_axis(v, g=g):
    return date(g.complex(v), g)
def plant_axis(v, g=g):
    return plant(g.complex(v), g)

In [34]:
variables = OrderedDict()

# on principal ramification path
#variables["branching_type"]= branching_type
variables['nb_module_axis'] = nb_modules_axis
variables['nb_visible_leaves_axis'] = nb_visible_leaves_axis # Nombre de feuille developpe
variables['nb_foliar_primordia_axis'] = nb_foliar_primordia_axis #Nombre de primordia foliaire
variables['nb_total_leaves_axis'] = nb_total_leaves_axis #Nombre total de feuille
variables['nb_vegetative_buds_axis']= nb_vegetative_buds_axis #
variables['nb_initiated_buds_axis']= nb_initiated_buds_axis
variables['nb_floral_buds_axis']= nb_floral_buds_axis
variables['nb_open_flowers_axis'] = nb_open_flowers_axis #Nombre de fleurs ouverte
variables['nb_aborted_flowers_axis'] = nb_aborted_flowers_axis #Nombre de fleurs avorté
variables['nb_total_flowers_axis'] = nb_total_flowers_axis #Nombre total de Fleurs
variables['nb_stolons_axis'] = nb_stolons_axis

#on whole ramification 
variables["nb_branch_crown_tree"]= nb_branching_tree # sum(No. visible modules if modules is_axis_root )
variables["nb_extenstion_crown_tree"]= nb_extension_tree # sum(No. visible module if modules nit is_axis_root)
variables["nb_total module_tree"]= nb_total_module_tree # sum(visible modules)
variables["nb_visible_leaves_tree"]= nb_visible_leaves_tree # sum(F, if visible module==T)
variables["nb_primordia_leaves_trees"]=nb_primordia_leaves_trees #sum(f, if visible module==T)
variables["nb_total_leaves_trees"]=nb_total_leaves_trees #sum(F & f , if visible module==T)
variables["nb_vegetative_buds_tree"]= nb_vegetative_buds_tree# sum(bt if stages in (None,'17','18','19') and visible module==F)
variables["nb_initiated_buds_tree"]= nb_initiated_buds_tree# sum(bt if stages=='A' and visible module==F)
variables["nb_floral_buds_tree"]= nb_floral_buds_tree# sum(ht if visible module==F)
variables["nb_inflorescence_tree"]= nb_inflorescences_tree# sum(HT if visible module==T)
variables["nb_open_flowers_tree"]= nb_open_flowers_tree# sum(g.property('Fleurs_ouverte') if visible module==T)
variables["nb_aborted_flowers_tree"]= nb_aborted_flowers_tree# sum(g.property('Fleurs_aborted') if visible module==T)
variables["nb_total_flowers_tree"]= nb_total_flowers_tree# sum(g.property('Fleurs_total') if visible module==T)
variables["nb_stolon_tree"]= nb_stolon_tree# sum(s if visible modules ==F)

# add supplemental variables
#variables No. Inflorescence
#variables No. Inflorescence crown bearing by ramification
#variables No Vegetative crown bearing by ramification
#variables sum initiated crown bearing by ramification
#variable sum floral crown bearing by ramification

#Global variable
global_variables = OrderedDict()
global_variables['genotype'] = genotype_axis # TODO
global_variables['date'] = date_axis # TODO
global_variables['plant'] = plant_axis

# Extract variables fo Inflorescences
inflo_variables = OrderedDict()
inflo_variables['nb_total_flowers_axis'] = nb_total_flowers 
inflo_variables['nb_open_flowers_axis'] = nb_open_flowers #Nombre de fleurs ouverte
inflo_variables['nb_aborted_flowers_axis'] = nb_aborted_flowers #Nombre de fleurs avorté


NameError: name 'nb_total_flowers' is not defined

In [35]:
final_seqs = []
# add if not ramif:
def is_inflo(g, vid):
    return True if g.label(vid) =='HT' else False

def filter_hamp_branch_crown(g, vid):
    """Extract first ramification node or inflo. """
    inflo = [c for c in g.children(vid) if is_inflo(g, c)]
    if inflo:
        return inflo[0]
    
    ramif = [c for c in g.children(vid) if (g.edge_type(c) =='+')]
    if not ramif:
        return None
    if len(ramif) > 1: 
        print vid, genotype(vid), 'line ', g.node(vid)._line, len(ramif)
    return ramif[0]


for trunk in trunks:
    seq = []
    for vid in trunk:
        vars = []
        # Extract the next vertex that is the root of a sub-tree
        bid = filter_hamp_branch_crown(g, vid)
        if bid is None:
            vars = [-1]
        else:
            vars = [branching_type(bid, g)]

        inflo = is_inflo(g, bid) 
        branch_id = branching(g, vid)
        for name in variables:
            if inflo and name in inflo_variables:

                f = inflo_variables[name]
                vars.append(f(g.complex(bid)))
                print 'INFLO ', f(g.complex(bid))
            elif branch_id is not None: 
                f = variables[name]
                vars.append(f(branch_id))
            else:
                vars.append(0)

            #vars.extend([f(branching(g, vid)) if branching(g, vid) is not None elif inflo_variables[name] if is_inflo(g, vid) else 0 for f in variables.values()])
        vars.extend([f(vid) for f in global_variables.values()])
        seq.append(vars)
    final_seqs.append(seq)

#[[f(branching(g, vid)) if branching(g, vid) is not None else -1 for f in variables.values()] for vid in trunk] for trunk in trunks]
#final_seqs                          

NameError: global name 'genotype' is not defined